<a href="https://colab.research.google.com/github/phantasy77/first_study/blob/main/0227_korean_sentiment_classification_my.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install konlpy

In [53]:
from tensorflow.keras import layers, models
import tensorflow_datasets as tfds
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [54]:
ds = tfds.load('huggingface:nsmc')
okt = Okt()

In [55]:
ds['train']

<_PrefetchDataset element_spec={'document': TensorSpec(shape=(), dtype=tf.string, name=None), 'id': TensorSpec(shape=(), dtype=tf.string, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None)}>

In [56]:
next(iter(ds['train']))

{'document': <tf.Tensor: shape=(), dtype=string, numpy=b'\xec\x95\x84 \xeb\x8d\x94\xeb\xb9\x99.. \xec\xa7\x84\xec\xa7\x9c \xec\xa7\x9c\xec\xa6\x9d\xeb\x82\x98\xeb\x84\xa4\xec\x9a\x94 \xeb\xaa\xa9\xec\x86\x8c\xeb\xa6\xac'>,
 'id': <tf.Tensor: shape=(), dtype=string, numpy=b'9976970'>,
 'label': <tf.Tensor: shape=(), dtype=int64, numpy=0>}

In [57]:
for data in ds['train']:
    print(data)
    print(data['document'])
    print((data['document'].numpy()))
    print((str(data['document'].numpy().decode())))
    break


{'document': <tf.Tensor: shape=(), dtype=string, numpy=b'\xec\x95\x84 \xeb\x8d\x94\xeb\xb9\x99.. \xec\xa7\x84\xec\xa7\x9c \xec\xa7\x9c\xec\xa6\x9d\xeb\x82\x98\xeb\x84\xa4\xec\x9a\x94 \xeb\xaa\xa9\xec\x86\x8c\xeb\xa6\xac'>, 'id': <tf.Tensor: shape=(), dtype=string, numpy=b'9976970'>, 'label': <tf.Tensor: shape=(), dtype=int64, numpy=0>}
tf.Tensor(b'\xec\x95\x84 \xeb\x8d\x94\xeb\xb9\x99.. \xec\xa7\x84\xec\xa7\x9c \xec\xa7\x9c\xec\xa6\x9d\xeb\x82\x98\xeb\x84\xa4\xec\x9a\x94 \xeb\xaa\xa9\xec\x86\x8c\xeb\xa6\xac', shape=(), dtype=string)
b'\xec\x95\x84 \xeb\x8d\x94\xeb\xb9\x99.. \xec\xa7\x84\xec\xa7\x9c \xec\xa7\x9c\xec\xa6\x9d\xeb\x82\x98\xeb\x84\xa4\xec\x9a\x94 \xeb\xaa\xa9\xec\x86\x8c\xeb\xa6\xac'
아 더빙.. 진짜 짜증나네요 목소리


In [58]:
raw_document = []
label = []
for idx, data in enumerate(ds['train']):
    raw_document.append(str(data['document'].numpy().decode()))
    label.append(data['label'].numpy())
    print(raw_document)
    print(label)
    if idx == 10:
        break



['아 더빙.. 진짜 짜증나네요 목소리']
[0]
['아 더빙.. 진짜 짜증나네요 목소리', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나']
[0, 1]
['아 더빙.. 진짜 짜증나네요 목소리', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '너무재밓었다그래서보는것을추천한다']
[0, 1, 0]
['아 더빙.. 진짜 짜증나네요 목소리', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '너무재밓었다그래서보는것을추천한다', '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정']
[0, 1, 0, 0]
['아 더빙.. 진짜 짜증나네요 목소리', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '너무재밓었다그래서보는것을추천한다', '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다']
[0, 1, 0, 0, 1]
['아 더빙.. 진짜 짜증나네요 목소리', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '너무재밓었다그래서보는것을추천한다', '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다', '막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.']
[0, 1, 0, 0, 1, 0]
['아 더빙.. 진짜 짜증나네요 목소리', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '너무재밓었다그래서보는것을추천한다', '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다', '막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.', '원작의 긴장감을 제

In [59]:
# 안녕하세요!, 안녕, 하세요 ,?, !
# 안녕, 하세요?,

In [60]:
def clean_str(string):
    string = re.sub(r"[^가-힣A-Za-z0-9(),!?]", " ", string)
    string = re.sub(r"\?", " ? ", string)
    string = re.sub(r"\!", " ! ", string)
    string = re.sub(r"\(", " ( ", string)
    string = re.sub(r"\)", " ) ", string)
    string = re.sub(r",", ' , ',string)
    string = re.sub(r"\s{2,}","",string)
    string = re.sub(r"\'{2,}","\'",string)
    string = re.sub(r"\'", "" , string)
    string = string.strip()
    return string.lower()


In [61]:
stop_words = ['의','이','가','은','들','는','과','와','하다','한']

In [62]:
raw_document = []
label = []
print(len(ds['train']))
for idx, data in enumerate(ds['train']):

    sentence = str(data['document'].numpy().decode())
    sentence = clean_str(sentence)
    #print(sentence)
    sentence = okt.morphs(sentence , stem = True)
    stop_removed_sentence = [word for word in sentence if not word in stop_words]

    if len(stop_removed_sentence) < 3:
        continue

    raw_document.append(stop_removed_sentence)
    label.append(data['label'].numpy())

    #print(raw_document)
    #print(label)
    #if idx == 10:
    #    break
    if idx % 1000 == 0 :
       print('+', end='')
    if idx % 10000 == 0:
       print(' %d\n' % idx)

print('\n')
print(len(raw_document))


150000
+

+++++++++

++++++++++++++++++

+++++++++

++++++++++

++++++++++

++++++++++

++++++++++

++++++++++

++++++++

++++++++++

++++++++++

+++++++++

++++++++++

+++++++138749


In [ ]:
#tmp = np.array(raw_document[:10])
raw_document[:10]

In [64]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(raw_document)

In [65]:
print(tokenizer.word_index) # 총 등장하는 단어의 수
print(tokenizer.word_counts) # 단어들이 몇 번 등장 했는지 확인

{'영화': 1, '보다': 2, '에': 3, '을': 4, '도': 5, ',': 6, '를': 7, '없다': 8, '이다': 9, '있다': 10, '?': 11, '좋다': 12, '너무': 13, '다': 14, '정말': 15, '!': 16, '되다': 17, '적': 18, '만': 19, '같다': 20, '재밌다': 21, '진짜': 22, '으로': 23, '점': 24, '로': 25, '않다': 26, '아니다': 27, '에서': 28, '만들다': 29, '나오다': 30, '평점': 31, '연기': 32, '것': 33, '내': 34, '최고': 35, '그': 36, '나': 37, '인': 38, '안': 39, '생각': 40, '스토리': 41, '못': 42, '왜': 43, '1': 44, '드라마': 45, '게': 46, '사람': 47, '감동': 48, '보고': 49, '이렇다': 50, '고': 51, '아깝다': 52, '말': 53, '더': 54, '때': 55, '!!': 56, '배우': 57, '감독': 58, '요': 59, '거': 60, '그냥': 61, '아': 62, '중': 63, '재미': 64, '까지': 65, '네': 66, '재미있다': 67, '주다': 68, '뭐': 69, '내용': 70, '좀': 71, '하고': 72, '자다': 73, '수': 74, '지루하다': 75, '가다': 76, '시간': 77, '쓰레기': 78, '재미없다': 79, '모르다': 80, '들다': 81, '지': 82, '싶다': 83, '10': 84, '그렇다': 85, '작품': 86, '알다': 87, '사랑': 88, '하나': 89, '볼': 90, '마지막': 91, '다시': 92, '2': 93, '잘': 94, '저': 95, '정도': 96, '오다': 97, '많다': 98, '처음': 99, '완전': 100, '장면': 101, '이건': 102, '액션': 

In [66]:
total_word_cnt = 0
min_cnt = 10
sparse_cnt = 0
unique_sparse_cnt = 0
for key, value in tokenizer.word_counts.items():
    # key 는 단어
    # value 등장 횟수
    total_word_cnt += value # 전체 등장 단어 수 체크
    if value < min_cnt:
        sparse_cnt += value # 희소 단어 총 갯수
        unique_sparse_cnt += 1

toal_cnt_unique = len(tokenizer.word_counts)
print(f'전체 단어의 수 {total_word_cnt}')
print(f'전체 단어의 수(유니크) {toal_cnt_unique}')
print(f'전체 등장 단어 중 희소 단어가 차지하는 비율 : {(sparse_cnt/total_word_cnt) * 100}')
print(f'희소 단어(유니크) / 전체 등장 단어수(유니크)  : {(unique_sparse_cnt /toal_cnt_unique) * 100}')




전체 단어의 수 1731904
전체 단어의 수(유니크) 47174
전체 등장 단어 중 희소 단어가 차지하는 비율 : 5.002875448061786
희소 단어(유니크) / 전체 등장 단어수(유니크)  : 80.1818798490694


In [67]:
tokenizer = Tokenizer(9000)
tokenizer.fit_on_texts(raw_document)

In [68]:
print(tokenizer.word_index) # 총 등장하는 단어의 수
print(tokenizer.word_counts) # 단어들이 몇 번 등장 했는지 확인

{'영화': 1, '보다': 2, '에': 3, '을': 4, '도': 5, ',': 6, '를': 7, '없다': 8, '이다': 9, '있다': 10, '?': 11, '좋다': 12, '너무': 13, '다': 14, '정말': 15, '!': 16, '되다': 17, '적': 18, '만': 19, '같다': 20, '재밌다': 21, '진짜': 22, '으로': 23, '점': 24, '로': 25, '않다': 26, '아니다': 27, '에서': 28, '만들다': 29, '나오다': 30, '평점': 31, '연기': 32, '것': 33, '내': 34, '최고': 35, '그': 36, '나': 37, '인': 38, '안': 39, '생각': 40, '스토리': 41, '못': 42, '왜': 43, '1': 44, '드라마': 45, '게': 46, '사람': 47, '감동': 48, '보고': 49, '이렇다': 50, '고': 51, '아깝다': 52, '말': 53, '더': 54, '때': 55, '!!': 56, '배우': 57, '감독': 58, '요': 59, '거': 60, '그냥': 61, '아': 62, '중': 63, '재미': 64, '까지': 65, '네': 66, '재미있다': 67, '주다': 68, '뭐': 69, '내용': 70, '좀': 71, '하고': 72, '자다': 73, '수': 74, '지루하다': 75, '가다': 76, '시간': 77, '쓰레기': 78, '재미없다': 79, '모르다': 80, '들다': 81, '지': 82, '싶다': 83, '10': 84, '그렇다': 85, '작품': 86, '알다': 87, '사랑': 88, '하나': 89, '볼': 90, '마지막': 91, '다시': 92, '2': 93, '잘': 94, '저': 95, '정도': 96, '오다': 97, '많다': 98, '처음': 99, '완전': 100, '장면': 101, '이건': 102, '액션': 

In [69]:
sequence = tokenizer.texts_to_sequences(raw_document)

In [70]:
drop_sequence_idx = [idx for idx , sentence in enumerate(sequence) if len(sentence) < 3 ]
obj_sequence = np.array(sequence, dtype='object')

In [71]:
np_sequence = np.delete(obj_sequence , drop_sequence_idx , axis=0)
np_label = np.delete(label , drop_sequence_idx, axis = 0)

In [72]:
len(np_sequence),len(np_label)

(136878, 136878)

In [73]:
pad_np_sequence = pad_sequences(np_sequence , maxlen = 82)

In [74]:
pad_np_sequence.shape

(136878, 82)

In [75]:
pad_np_sequence[:3]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          62,  477,   22,  280,  681],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,

In [76]:
model = models.Sequential()
model.add(layers.Embedding(9000, 100))
model.add(layers.LSTM(64))
model.add(layers.Dense(1, activation='sigmoid'))

In [77]:
model.compile(optimizer='adam' , loss='binary_crossentropy', metrics=['accuracy'])
model.fit(pad_np_sequence, np_label, epochs=2 , validation_split=0.2)

Epoch 1/2
3422/3422 [==============================] - 255s 69ms/step - loss: 0.3754 - accuracy: 0.8316 - val_loss: 0.3351 - val_accuracy: 0.8552
Epoch 2/2
3422/3422 [==============================] - 254s 74ms/step - loss: 0.2930 - accuracy: 0.8753 - val_loss: 0.3280 - val_accuracy: 0.8602


In [86]:
def decision(raw_test):
  test = clean_str(raw_test)
  test = okt.morphs(test , stem = True)
  stop_removed_test = [word for word in test if not word in stop_words]
  print(stop_removed_test)
  print([stop_removed_test])

  tokenized_test = tokenizer.texts_to_sequences([stop_removed_test])
  padded_test = pad_sequences(tokenized_test , maxlen = 82)
  answer = model.predict(padded_test)
  print(answer)
  if answer < 0.5:
    print('"', raw_test, '" is negative answer.')
  else:
    print('"', raw_test, '" is positive answer.')

test = '아 이 영화 진짜 재미없네'
decision(test)
test = '아 이 영화 짱이다.'
decision(test)


['아', '영화', '진짜', '재미없다']
[['아', '영화', '진짜', '재미없다']]
1/1 [==============================] - 0s 24ms/step
[[0.01249121]]
" 아 이 영화 진짜 재미없네 " is negative answer.
['아', '영화', '짱', '이다']
[['아', '영화', '짱', '이다']]
1/1 [==============================] - 0s 25ms/step
[[0.8880675]]
" 아 이 영화 짱이다. " is positive answer.
